In [ ]:
pip install numpy

In [ ]:
import numpy as np

## Implementación de similitud por diagramas de transición

In [ ]:
def get_file(file):
    return open(file, 'r', encoding='utf-8', errors='ignore').read()

In [ ]:
def get_words(text):
  file = open(text, "r", encoding = 'utf-8', errors ='ignore')
  words = file.read()
  file.close()

  punctuation = ["_", "-", ".", ":", "," , ";", "(", ")", "?", "¿", "¡", "!"]
  for i in punctuation:
      words = words.replace(i, " ")

  words = words.lower()
  words = words.replace('\n', ' ')

  return words

In [ ]:
def split_words(text1, text2):
  splitted_words = list(set(text1.split() + text2.split()))
  return splitted_words

In [ ]:
def create_transition_matrix(text, words):
  # Inicializar la matriz de transición
  matrix = np.zeros((len(words), len(words)))

  # Dividir el texto en palabras
  word_list = text.split()

  # Calcular las frecuencias de las transiciones entre palabras
  for i in range(len(word_list) - 1):
      current_word = word_list[i]
      next_word = word_list[i + 1]

      current_index = words.index(current_word)
      next_index = words.index(next_word)

      matrix[current_index][next_index] += 1

  # Normalizar las filas de la matriz, evitando divisiones por cero
  row_sums = matrix.sum(axis=1)
  nonzero_rows = np.nonzero(row_sums)
  matrix_normalized = np.zeros_like(matrix)
  matrix_normalized[nonzero_rows, :] = matrix[nonzero_rows, :] / row_sums[nonzero_rows, np.newaxis]

  return matrix_normalized

In [ ]:
def cos_ang(mat1, mat2):
  mat2T = np.transpose(mat2)
  c = np.dot(mat2T, mat1)
  prod_int = np.trace(c)
  normA = np.sqrt(np.trace((np.dot(np.transpose(mat1), mat1))))
  normB = np.sqrt(np.trace((np.dot(np.transpose(mat2), mat2))))

  print("prod_int = %.4f" %prod_int)
  print("normA = %.4f" %normA)
  print("normB = %.4f" %normB)

  return prod_int / (normA * normB)

## Similitud entre dos códigos hechos en C. ¿Qué preprocesamiento tendrías que hacer con los códigos antes de compararlos? (Por ejemplo, a = b + c y r = m + n se tomarían como completamente disintos. ¿Cómo podrías evitar eso?

In [ ]:
import re

def remove_comments(text):
    # Remove all block comments (/* */)
    text = re.sub(r'/\*.*?\*/', '', text, flags=re.DOTALL)
    # Remove all line comments (//)
    text = re.sub(r'//.*', '', text)
    # Remove all Python line comments (#)
    text = re.sub(r'#.*', '', text)
    # Remove all Python block comments (''' ''')
    text = re.sub(r"'''(.*?)'''", '', text, flags=re.DOTALL)
    # Remove all Python block comments (""" """)
    text = re.sub(r'"""(.*?)"""', '', text, flags=re.DOTALL)
    
    # Split the text into lines, clean empty lines
    lines = text.split('\n')
    cleaned_lines = [line for line in lines if line.strip() != '']
    
    cleaned_text = '\n'.join(cleaned_lines)
    return cleaned_text

# Ejemplo de uso
code = """/* Block comment */
int x = 0; // Line comment

x++; /* Another block
comment */

System.out.println(x); // Print x

# This is a Python comment
y = 2 # Another Python comment

''' Python block comment
spanning multiple lines '''
z = 3; // C style line comment

"""
cleaned_code = remove_comments(code)
print(cleaned_code)


int x = 0; 
x++; 
System.out.println(x); 
y = 2 
z = 3; 


In [34]:
def similitud(text1, text2):
  data1 = remove_comments(text1)
  data2 = remove_comments(text2)

  split_data = split_words(data1, data2)

  mat1 = create_transition_matrix(data1, split_data)
  mat2 = create_transition_matrix(data2, split_data)
  cos = cos_ang(mat1, mat2)

  print("La similitud de los textos es de: %.4f" %cos)

  return cos # Debe retornar un valor %cos


In [35]:
# print("La similitud de los textos es de: %.4f" %cos)
print(similitud("a c b a d c", "b a c a b d b a b"))

prod_int = 0.8333
normA = 1.8708
normB = 1.7638
La similitud de los textos es de: 0.2525
0.25253813613805265
